<a href="https://colab.research.google.com/github/otitamario/sp-pa-gep/blob/main/experiments/Example5_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Clone the repository into Colab runtime
!git clone https://github.com/otitamario/sp-pa-gep.git

# Move into repo root
%cd sp-pa-gep

# Make sure Python sees the project root
import sys
sys.path.append(".")

Cloning into 'sp-pa-gep'...
remote: Enumerating objects: 253, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 253 (delta 46), reused 23 (delta 23), pack-reused 184 (from 1)
Receiving objects: 100% (253/253), 4.38 MiB | 29.52 MiB/s, done.
Resolving deltas: 100% (114/114), done.
/content/sp-pa-gep/sp-pa-gep


In [8]:
import os
# =========================
# OUTPUT DIRECTORY (save plots here)
# =========================
FIGDIR = "figures"
os.makedirs(FIGDIR, exist_ok=True)

In [9]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import src.benchkit as bk
import time

# =========================
# OUTPUT DIRECTORY (save plots here)
# =========================
FIGDIR = "figures"
os.makedirs(FIGDIR, exist_ok=True)

# =========================
# Rosen–Suzuki data (Example 1)
# =========================
x_star = np.array([0.0, 1.0, 2.0, -1.0])

def error_fn(x):
    return float(np.linalg.norm(x - x_star))

def phi(x):
    return (x[0]**2 + x[1]**2 + 2*x[2]**2 + x[3]**2
            - 5*x[0] - 5*x[1] - 21*x[2] + 7*x[3])

def grad_phi(x):
    return np.array([
        2*x[0] - 5,
        2*x[1] - 5,
        4*x[2] - 21,
        2*x[3] + 7
    ])

# Constraints in the form g_i(x) <= 0 (as in the paper)
def g1_raw(x):
    return x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2 + x[0] - x[1] + x[2] - x[3] - 8

def g2_raw(x):
    return x[0]**2 + 2*x[1]**2 + x[2]**2 + 2*x[3]**2 - x[0] - x[3] - 10

def g3_raw(x):
    return 2*x[0]**2 + x[1]**2 + x[2]**2 + 2*x[0] - x[1] - x[3] - 5

def constr_violation(x):
    """Max violation of g_i(x) <= 0 constraints (0 means feasible)."""
    v = max(g1_raw(x), g2_raw(x), g3_raw(x))
    return float(max(0.0, v))

# scipy 'ineq' expects c(x) >= 0, so use -g_i(x) >= 0

constraints = [
    {'type': 'ineq', 'fun': lambda x: -g1_raw(x),
     'jac': lambda x: -np.array([2*x[0]+1, 2*x[1]-1, 2*x[2]+1, 2*x[3]-1])},
    {'type': 'ineq', 'fun': lambda x: -g2_raw(x),
     'jac': lambda x: -np.array([2*x[0]-1, 4*x[1], 2*x[2], 4*x[3]-1])},
    {'type': 'ineq', 'fun': lambda x: -g3_raw(x),
     'jac': lambda x: -np.array([4*x[0]+2, 2*x[1]-1, 2*x[2], -1])},
]

# =========================
# Projection onto C
# =========================
def proj_C(z, x_init=None):
    """
    Best-effort projection onto C:
      P_C(z) = argmin_{y in C} 0.5||y-z||^2
    Returns (p, ok) where ok=True if some run reported success.
    """
    def obj(y):
        d = y - z
        return 0.5*np.dot(d, d)

    # candidate initial points
    inits = []
    if x_init is not None:
        inits.append(np.array(x_init, dtype=float))
    inits.append(np.array(z, dtype=float))
    inits.append(np.zeros(4, dtype=float))
    inits.append(x_star.copy())

    best = None
    best_val = np.inf
    any_success = False

    for init in inits:
        res = minimize(
            obj, init, method="SLSQP", constraints=constraints,
            options={"ftol": 1e-9, "maxiter": 1000}
        )
        y = res.x
        val = obj(y)
        if val < best_val:
            best_val = val
            best = y
        if res.success:
            any_success = True
            # return first success immediately (or keep searching if you prefer)
            return y, True

    return best, any_success
# Residual R(x) = ||x - P_C(x - grad phi(x))||
def residual_R(x):
    z = x - grad_phi(x)
    p, ok = proj_C(z, x_init=x)
    # Even if ok=False, we still compute a residual with best candidate p.
    return np.linalg.norm(x - p), ok


# =========================
# Resolvent for EP: u_n = argmin_{y in C} phi(y) + (1/(2r))||y - x_n||^2
# =========================
def resolvent(xn, r=1.0, x_init=None, ftol=1e-9, maxiter=1000):
    """
    Resolvent subproblem:
        u = argmin_{y in C}  phi(y) + (1/(2r))||y - xn||^2
    Solved approximately with SLSQP.

    Returns:
        u, ok, nit, hit_max, constraint_violation(u)
    """
    def obj(y):
        d = y - xn
        return phi(y) + 0.5 / r * np.dot(d, d)

    # candidate initial points (in order)
    inits = []
    if x_init is not None:
        inits.append(np.array(x_init, dtype=float))
    inits.append(np.array(xn, dtype=float))
    inits.append(np.zeros(4, dtype=float))
    inits.append(x_star.copy())

    # also try projection of xn onto C
    pxn, ok_pxn = proj_C(xn, x_init=np.zeros(4))
    if ok_pxn and pxn is not None:
        inits.append(np.array(pxn, dtype=float))

    best = None
    best_val = np.inf
    best_res = None

    for init in inits:
        res = minimize(
            obj, init, method="SLSQP", constraints=constraints,
            options={"ftol": ftol, "maxiter": maxiter}
        )
        y = res.x
        val = obj(y)

        if val < best_val:
            best_val = val
            best = y
            best_res = res

        if res.success:
            nit = int(getattr(res, "nit", -1))
            cv = constr_violation(y)
            return y, True, nit, (nit == maxiter), cv

    # If all failed, return best found but mark as False
    if best is None:
        best = np.array(xn, dtype=float)

    nit = int(getattr(best_res, "nit", -1)) if best_res is not None else -1
    cv = constr_violation(best)
    return best, False, nit, (nit == maxiter), cv
# =========================
# SPPA / WPPA runners with logging
# =========================
def make_sppa_step(r):
    def step_fn(x, k):
        alpha = 1.0 / (k + 2.0)

        u, ok, nit, hit_max, cv = resolvent(x, r=r, x_init=x)
        residual = float(np.linalg.norm(x - u))  # <-- correct proximal residual

        x_new = alpha * u_anchor + (1.0 - alpha) * u

        return x_new, {
            "u": u,
            "residual": residual,
            "ok": bool(ok),
            "inner_iters": nit,
            "inner_hit_max": bool(hit_max),
            "constraint_violation": cv,
        }
    return step_fn


def make_wppa_step(r):
    def step_fn(x, k):
        alpha = 1.0 / (k + 2.0)

        u, ok, nit, hit_max, cv = resolvent(x, r=r, x_init=x)
        residual = float(np.linalg.norm(x - u))

        x_new = alpha * x + (1.0 - alpha) * u

        return x_new, {
            "u": u,
            "residual": residual,
            "ok": bool(ok),
            "inner_iters": nit,
            "inner_hit_max": bool(hit_max),
            "constraint_violation": cv,
        }
    return step_fn

In [10]:
import src.benchkit as bk
import numpy as np

r = 1.0
maxit = 200
stop_tol_step = 0.0

# initial point and anchor
x0 = np.array([1.8, 1.8, 1.8, 1.8], dtype=float)
u_anchor = x0.copy()   # Halpern anchor = x0 (as you’re using)

logs_sppa, sum_sppa = bk.run(
    method_name="SPPA",
    x0=x0,
    max_iter=maxit,
    stop_tol_step=stop_tol_step,
    step_fn=make_sppa_step(r),
    residual_fn=None,
    error_fn=error_fn,
)

logs_wppa, sum_wppa = bk.run(
    method_name="WPPA",
    x0=x0,
    max_iter=maxit,
    stop_tol_step=stop_tol_step,
    step_fn=make_wppa_step(r),
    residual_fn=None,
    error_fn=error_fn,
)

In [11]:
bk.make_standard_plots(
    logs_by_method={"SPPA": logs_sppa, "WPPA": logs_wppa},
    outdir="figures",
    tag="ex54_rosen_suzuki",
    plot_residual=True,
    plot_error=True,
)

print(bk.latex_tables_split(
    [sum_sppa, sum_wppa],
    caption_perf="Computational performance for Experiment 4.",
    label_perf="tab:ex54_perf",
    caption_acc="Accuracy and feasibility metrics for Experiment 4.",
    label_acc="tab:ex54_acc",
))

AttributeError: module 'src.benchkit' has no attribute 'latex_tables_split'